# Anomali Detection using Isolation Forest

**Setup**

In [74]:
from azureml.core import Workspace, Datastore, Dataset

from azureml.core.experiment import Experiment

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from pandas.plotting import parallel_coordinates

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

from sklearn.ensemble import IsolationForest

from azureml.core import Workspace, Datastore, Dataset
from azureml.core.model import Model

from azureml.core.experiment import Experiment

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from pandas.plotting import parallel_coordinates

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

import joblib



In [75]:
#ws = Workspace.from_config("../config/config")
ws = Workspace.get(name="AzureML02WP",
               subscription_id='5e20a2e2-7522-4cb5-957f-ce42d9c2abfd',
               resource_group='AzureML02')
ws
az_store = Datastore.get(ws, 'blob01')
az_dataset = Dataset.get_by_name(ws, "petrosani")
#az_dataset_Turda = Dataset.get_by_name(ws, "Turda_01")
az_default_store = ws.get_default_datastore()
df = az_dataset.to_pandas_dataframe()
#df_Turda_ = az_dataset_Turda.to_pandas_dataframe()

In [76]:
experiment = Experiment(workspace=ws, name='AnomaliDetection')
new_run = experiment.start_logging()

data_colums = df.select_dtypes(include='number').columns

In [77]:
df_clean = df.drop(['Temp_Id', 'Location_Id'], 1)
df_clean

C:\Users\OguzX\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,Date_Id,CoolingSource_Id,Temp_Out,Temp_Floor,Temp_Room,FanSpeed,ControllerStateNumber
0,2020-12-31 23:00:00,0,5.6,16.3,24.7,57,7.0
1,2020-12-31 23:02:00,0,5.6,16.0,24.8,58,7.0
2,2020-12-31 23:04:00,0,5.6,15.7,24.9,59,7.0
3,2020-12-31 23:06:00,0,5.8,15.4,24.7,57,7.0
4,2020-12-31 23:08:00,0,5.5,15.1,24.7,58,7.0
...,...,...,...,...,...,...,...
333624,2022-05-18 14:20:00,0,17.7,18.3,26.2,63,6.0
333625,2022-05-18 14:22:00,0,17.8,18.4,26.2,63,6.0
333626,2022-05-18 14:24:00,0,17.9,18.3,26.2,63,6.0
333627,2022-05-18 14:26:00,0,18.9,18.8,26.2,63,6.0


# Isolation Forest fitting & adjusting

In [78]:
model=IsolationForest(n_estimators=500, max_samples='auto', contamination=float(0.1),max_features=1.0)
fitted_model = model.fit(df_clean[['FanSpeed', 'Temp_Room', 'Temp_Out', 'Temp_Floor']])


c:\Users\OguzX\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"


In [79]:
# joblib dump
model_files = 'outputs/isolationforest.pkl'
joblib.dump(value=[data_colums,fitted_model], filename=model_files)

['outputs/isolationforest.pkl']

**Assign labels and prediction values**

In [80]:
df_clean['scores']=model.decision_function(df_clean[['FanSpeed', 'Temp_Room', 'Temp_Out', 'Temp_Floor']])
df_clean['anomaly']=model.predict(df_clean[['FanSpeed', 'Temp_Room', 'Temp_Out', 'Temp_Floor']])
df_clean.head(20)

,Date_Id,CoolingSource_Id,Temp_Out,Temp_Floor,Temp_Room,FanSpeed,ControllerStateNumber,scores,anomaly
0,2020-12-31 23:00:00,0,5.6,16.3,24.7,57,7.0,0.173155,1
1,2020-12-31 23:02:00,0,5.6,16.0,24.8,58,7.0,0.177236,1
2,2020-12-31 23:04:00,0,5.6,15.7,24.9,59,7.0,0.177439,1
3,2020-12-31 23:06:00,0,5.8,15.4,24.7,57,7.0,0.170077,1
4,2020-12-31 23:08:00,0,5.5,15.1,24.7,58,7.0,0.163001,1
5,2020-12-31 23:10:00,0,5.3,14.8,24.6,56,7.0,0.154779,1
6,2020-12-31 23:12:00,0,5.6,14.5,24.6,56,7.5,0.150753,1
7,2020-12-31 23:14:00,0,5.5,14.2,25.7,67,7.5,0.138760,1
8,2020-12-31 23:16:00,0,5.5,14.1,25.7,66,7.5,0.146209,1
9,2020-12-31 23:18:00,0,5.6,14.2,25.7,67,7.5,0.138879,1


**Observe total anomalies**

In [81]:
anomaly=df_clean.loc[df_clean['anomaly']==-1]
anomaly_index=list(anomaly.index)
print(anomaly[['FanSpeed', 'Temp_Out', 'Temp_Floor','anomaly', 'scores']])

        FanSpeed  Temp_Out  Temp_Floor  anomaly    scores
968           39      -2.3        19.0       -1 -0.054750
969           16      -2.3        19.2       -1 -0.097976
970           16      -1.9        18.8       -1 -0.097271
971           16      -1.3        18.6       -1 -0.095486
972           16      -0.5        18.7       -1 -0.093402
...          ...       ...         ...      ...       ...
332946         0      22.0        25.3       -1 -0.057146
332947         0      21.3        27.4       -1 -0.079308
332948         0      21.1        27.8       -1 -0.088238
332949         0      20.9        28.1       -1 -0.101229
332950        70      19.9        26.3       -1 -0.070942

[33363 rows x 5 columns]


**Isolate anomalies based on conditions and other criteria**

In [82]:
#outliers_counter = len(df[df['FanSpeed'] > 80 & df['Temp_Room'] > 20 & (df['Temp_Room'] > (df['Temp_Out'] + 10))])
outliers_counter = len(df_clean[df_clean['scores'] < 0 ])
outliers_counter

33363

In [83]:
scores=df_clean.loc[((df_clean['scores'] < 0) & (df_clean['FanSpeed'] > 0) & (df_clean['ControllerStateNumber'] > 0) \
    & (df_clean['Temp_Out'] < 26)
    & (df_clean['ControllerStateNumber'] != 71.0) 
    & (df_clean['ControllerStateNumber'] != 71.1)
    & (df_clean['ControllerStateNumber'] != 72.0)
    & (df_clean['ControllerStateNumber'] != 73.0)
    & (df_clean['ControllerStateNumber'] != 74.0)
    & (df_clean['ControllerStateNumber'] != 74.4)
    & (df_clean['ControllerStateNumber'] != 71.4)
    )]
    
scores

,Date_Id,CoolingSource_Id,Temp_Out,Temp_Floor,Temp_Room,FanSpeed,ControllerStateNumber,scores,anomaly
983,2021-01-02 07:46:00,1,-0.9,18.0,21.1,22,9.5,-0.087314,-1
984,2021-01-02 07:48:00,1,-1.9,16.6,20.3,15,9.5,-0.078294,-1
985,2021-01-02 07:50:00,1,-2.2,16.3,19.9,15,9.5,-0.076239,-1
986,2021-01-02 07:52:00,1,-2.2,15.9,19.5,15,9.5,-0.074799,-1
987,2021-01-02 07:54:00,0,-2.6,10.3,21.6,26,9.1,-0.098556,-1
...,...,...,...,...,...,...,...,...,...
332912,2022-05-17 14:31:00,2,21.7,21.3,28.6,79,15.1,-0.019737,-1
332913,2022-05-17 14:33:00,2,22.7,21.3,28.6,79,15.1,-0.024443,-1
332914,2022-05-17 14:35:00,2,21.7,21.3,28.6,79,15.1,-0.019737,-1
332915,2022-05-17 14:37:00,1,21.5,23.3,27.7,70,24.2,-0.007050,-1


In [84]:
scores=df_clean.loc[((df_clean['scores'] < -0.05) & (df_clean['Temp_Out'] < 25) )]
scores[['anomaly']].count()

anomaly    4692
dtype: int64

In [85]:
scaled = df_clean.loc[((df_clean['scores'] < 0) & (df_clean['FanSpeed'] > 0) & (df_clean['ControllerStateNumber'] > 0) \
                        & (df_clean['Temp_Out'] < 26)
                        & (df_clean['ControllerStateNumber'] != 71.0) 
                        & (df_clean['ControllerStateNumber'] != 71.1)
                        & (df_clean['ControllerStateNumber'] != 72.0)
                        & (df_clean['ControllerStateNumber'] != 73.0)
                        & (df_clean['ControllerStateNumber'] != 74.0)
                        & (df_clean['ControllerStateNumber'] != 74.4)
                        & (df_clean['ControllerStateNumber'] != 71.4)
                        )]

scaled = scaled[['scores', 'anomaly', 'FanSpeed']]
scaler = StandardScaler()
X = StandardScaler().fit_transform(scaled)
X

array([[-2.77516409,  0.        , -2.74539731],
       [-2.36352726,  0.        , -3.13127368],
       [-2.26978274,  0.        , -3.13127368],
       ...,
       [ 0.3085927 ,  0.        ,  0.39673883],
       [ 0.88750509,  0.        , -0.09938793],
       [-2.0280352 ,  0.        , -0.09938793]])

# Accuracy calculations

In [86]:
accuracy = 100*list(df_clean['anomaly']).count(-1)/(outliers_counter)
print("Accuracy percentage:", accuracy)

Accuracy percentage: 100.0


In [87]:
result = (scores['anomaly'].count()/df_clean['anomaly'].count()) * 100
print("Percentage of anomalies:", result)

Percentage of anomalies: 1.4063525652746014


# Isolation Forest for March, April and May

In [88]:
marchstart = '2021-03-01'
marchend = '2021-03-31'
#
aprilstart = '2021-04-01'
aprilend = '2021-04-30'
#
maystart = '2021-05-01'
mayend = '2021-05-31'

# create features
maskmarch = (df['Date_Id'] > marchstart) & (df['Date_Id'] <= marchend)
maskapril = (df['Date_Id'] > aprilstart) & (df['Date_Id'] <= aprilend)
maskmay = (df['Date_Id'] > maystart) & (df['Date_Id'] <= mayend)



df_march = df.loc[maskmarch]
df_april = df.loc[maskapril]
df_may = df.loc[maskmay]


In [89]:
# march
model_march = IsolationForest(n_estimators=100, max_samples='auto', contamination=float(0.1),max_features=1.0)
model_march.fit(df_march[['FanSpeed', 'Temp_Room', 'Temp_Out', 'Temp_Floor']])

c:\Users\OguzX\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"


IsolationForest(contamination=0.1)

In [90]:
# april
model_april = IsolationForest(n_estimators=100, max_samples='auto', contamination=float(0.1),max_features=1.0)
model_april.fit(df_april[['FanSpeed', 'Temp_Room', 'Temp_Out', 'Temp_Floor']])


c:\Users\OguzX\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"


IsolationForest(contamination=0.1)

In [91]:

# may
model_may = IsolationForest(n_estimators=100, max_samples='auto', contamination=float(0.1),max_features=1.0)
model_may.fit(df_may[['FanSpeed', 'Temp_Room', 'Temp_Out', 'Temp_Floor']])

c:\Users\OguzX\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"


IsolationForest(contamination=0.1)

In [92]:
# march
df_march['scores']=model_march.decision_function(df_march[['FanSpeed', 'Temp_Room', 'Temp_Out', 'Temp_Floor']])
df_march['anomaly']=model_march.predict(df_march[['FanSpeed', 'Temp_Room', 'Temp_Out', 'Temp_Floor']])

C:\Users\OguzX\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\OguzX\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [93]:
# april
df_april['scores']=model_april.decision_function(df_april[['FanSpeed', 'Temp_Room', 'Temp_Out', 'Temp_Floor']])
df_april['anomaly']=model_april.predict(df_april[['FanSpeed', 'Temp_Room', 'Temp_Out', 'Temp_Floor']])

C:\Users\OguzX\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\OguzX\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [94]:
# may
df_may['scores']=model_may.decision_function(df_may[['FanSpeed', 'Temp_Room', 'Temp_Out', 'Temp_Floor']])
df_may['anomaly']=model_may.predict(df_may[['FanSpeed', 'Temp_Room', 'Temp_Out', 'Temp_Floor']])

C:\Users\OguzX\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\OguzX\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [95]:
df_total = pd.DataFrame()

df_total[['March-FanSpeed', 'March-Temp_Room', 'March-Temp_Out', 'March-Temp_Floor', 'March-scores', 'March-anomaly']] = df_march[['FanSpeed', 'Temp_Room', 'Temp_Out', 'Temp_Floor', 'scores', 'anomaly']]
df_total[['April-FanSpeed', 'April-Temp_Room', 'April-Temp_Out', 'April-Temp_Floor', 'April-scores', 'April-anomaly']] = df_april[['FanSpeed', 'Temp_Room', 'Temp_Out', 'Temp_Floor', 'scores', 'anomaly']]
df_total[['May-FanSpeed', 'May-Temp_Room', 'May-Temp_Out', 'May-Temp_Floor', 'May-scores', 'May-anomaly']] = df_may[['FanSpeed', 'Temp_Room', 'Temp_Out', 'Temp_Floor', 'scores', 'anomaly']]


In [96]:
df_total

,March-FanSpeed,March-Temp_Room,March-Temp_Out,March-Temp_Floor,March-scores,March-anomaly,April-FanSpeed,April-Temp_Room,April-Temp_Out,April-Temp_Floor,April-scores,April-anomaly,May-FanSpeed,May-Temp_Room,May-Temp_Out,May-Temp_Floor,May-scores,May-anomaly
42400,67,25.7,2.9,14.1,0.004191,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42401,66,25.6,2.9,14.3,0.043701,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42402,66,25.6,3.1,14.5,0.058640,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42403,66,25.6,3.3,14.9,0.072078,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42404,68,25.8,3.4,15.2,0.009262,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63931,52,24.2,2.2,16.3,0.100897,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63932,51,24.1,2.3,16.1,0.088606,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63933,53,24.3,2.3,15.7,0.102249,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63934,52,24.2,2.1,15.3,0.096814,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Deployment configurations

In [97]:
new_run.complete()

In [98]:
new_run.register_model(model_path='outputs/isolationforest.pkl',
                       model_name='ISOF02', 
                       tags={'source': 'SDK-Run', 'algorithm': 'Isolation Forest'},
                       description='Isolation Forest algorithm for anomali predictions')

Model(workspace=Workspace.create(name='AzureML02WP', subscription_id='5e20a2e2-7522-4cb5-957f-ce42d9c2abfd', resource_group='AzureML02'), name=ISOF02, id=ISOF02:1, version=1, tags={'source': 'SDK-Run', 'algorithm': 'Isolation Forest'}, properties={})

In [99]:
# environment
from azureml.core import Environment
from azureml.core.environment import CondaDependencies

myenv = Environment(name='ClusterPetroEnvironment_02')


python_packages = ['pandas', 'joblib']
for package in python_packages:
    myenv.python.conda_dependencies.add_pip_package(package)

myenv_dep = CondaDependencies.create(conda_packages=['scikit-learn', 'pip'],
                                     pip_packages=['azureml-defaults'])

myenv.python.conda_dependencies = myenv_dep

myenv.register(ws)



{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20220412.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "ClusterPetroEnvironment_02",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "

In [100]:
from azureml.core.compute import AksCompute
from azureml.core.compute import ComputeTarget

cluster_name = 'aks-anomali-001'

if cluster_name not in ws.compute_targets:
    aks_config = AksCompute.provisioning_configuration(location='westeurope',
                                                    vm_size='Standard_A2m_v2',
                                                    agent_count=1,
                                                    cluster_purpose='DevTest')

    production_cluster = ComputeTarget.create(ws, cluster_name, aks_config)

    production_cluster.wait_for_completion(show_output=True)
else:
        production_cluster = ws.compute_targets[cluster_name]

In [101]:
from azureml.core.model import InferenceConfig

_inference_config = InferenceConfig(environment=myenv, 
                                   entry_script='score_script_anomali.py',
                                   source_directory='../source')

In [102]:
from azureml.core.webservice import AksWebservice

deploy_config = AksWebservice.deploy_configuration(cpu_cores=1,
                                                   memory_gb=1)

In [ ]:
from azureml.core.model import Model

# models to deploy to service
models = ws.models['ISOF02']

service = Model.deploy(workspace=ws,
                       name='isof-service-03',
                       models=[models],
                       inference_config=_inference_config,
                       deployment_config=deploy_config,
                       deployment_target=production_cluster
                       )

service.wait_for_deployment(show_output=True)